# Tag salinity particles

Tag sea surface salinity for initial and final time of every simulation.

In [24]:
import numpy as np
import xarray as xr
from pandas import read_csv
import matplotlib.pyplot as plt
from matplotlib import dates as mdates
from datetime import datetime
from scipy.interpolate import RegularGridInterpolator
from os import path
# cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature
# plots style
import scienceplots
plt.style.use('science')
plt.rcParams['figure.constrained_layout.use'] = True

All directories

In [25]:
# inputs
pathSMOS = '../data/SSS_data/SMOS_L3_DEBIAS_LOCEAN_v8/debiasedSSS_09days_v8/*.nc'
pathVEL = '../data/velocity_data/cmems_obs-sl_glo_phy-ssh_my_allsat-l4-duacs-0.25deg_P1D_1701257133382.nc'
mainpathSIM = "../outputs/" # main path of the simulation
pathSIM = mainpathSIM + "sim/" # path of simulation outputs

# outputs
pathADVSSS = mainpathSIM + "sim_tagged/" # path to store advected sss
pathPLOT = "../plots/sim/tag_sss/" # path for plots

Parameters

In [26]:
# running days
rds = [4,8]

# save figures?
save = True 

# domain limits
time_lim = ["2016-04-08", "2016-05-12"]
lonmin, lonmax = -70, -35
latmin, latmax = -50, -20

Functions used for plots

In [27]:
def map_plot(ax,lonmin,lonmax,latmin,latmax):
    """Decorations for plotting geopositional data"""
    ax.set_extent([lonmin, latmin, lonmax, latmax], crs=ccrs.PlateCarree())
    ax.add_feature(cfeature.LAND,facecolor="silver")
    ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True, linewidth=.5, color="k", alpha=.5)

def plot_insitu(ds_TSG,ax,variable,cmap,vmin,vmax):
    """Plot in-situ data. Scatter with cmap of salinity"""
    ax.scatter(ds_TSG["longitude"],ds_TSG["latitude"],s=24,c='w')
    plot = ax.scatter(ds_TSG["longitude"],ds_TSG["latitude"],c=ds_TSG[variable],cmap=cmap,vmin=vmin,vmax=vmax,s=20,label="")
    return plot

Open SMOS dataset with domain limits

In [28]:
# SMOS SSS L3 maps generated by CATDS CEC LOCEAN (9 days)
dsSMOS = xr.open_mfdataset(pathSMOS)
dsSMOS = xr.open_mfdataset(pathSMOS).sel(time=slice(time_lim[0],time_lim[1]),lon=slice(lonmin,lonmax),lat=slice(latmin,latmax))

Get simulation dates from in-situ dataset to obtain the simulation dates

In [29]:
# extract SMOS and TSG time dimension
timeSMOS = np.array(dsSMOS.time.values,dtype="datetime64[s]")

### Loop for each simulation
Open dataset of every simulation and extract positions (lon,lat) of the particles at initial time T0 and final time TF. Then interpolate SMOS SSS values at initial positions (T0) to tag particles salinity and then obtain the final position of these particles by using the final positions (TF) and the salinity interpolated values

In [30]:
for date in timeSMOS:
    
    date_obj = date.item()

    for rd in rds:

          filename = "sim"+ str(date)[:4] + str(date)[5:7] + str(date)[8:10] +"_" + '%02i'% rd + "days.zarr"

          print("")
          print("Openning ... " + filename)

          """Open simulation data"""

          ind_tf = 0 # first observation, beginning of the simulation at TF
          ind_ti = -1 # last observation, end of simulation at T0

          ds = xr.open_dataset(pathSIM+"advection" + '%02i'% rd +"days/"+filename,engine='zarr',chunks={})

          # time_ti in python format
          time_ini = ds.time[:,ind_ti].values[~np.isnan(ds.time[:,ind_ti].values)][0] # from seconds to days
          time_fin = ds.time[:,ind_tf].values[~np.isnan(ds.time[:,ind_tf].values)][0] 

          print('')
          print('ini date...',time_ini)
          print('')

          """Interpolate the corresponding SSS SMOS data to each particle initial position"""

          interpolate = RegularGridInterpolator((dsSMOS.sel(time=time_ini,method='nearest').lon.values.ravel(), 
                                             dsSMOS.sel(time=time_ini,method='nearest').lat.values.ravel())
                                             ,dsSMOS.sel(time=time_ini,method='nearest').SSS.values.T,bounds_error=False)
          
          sss_int = interpolate((ds.lon[:,ind_ti].values,ds.lat[:,ind_ti].values),'linear')

          smin = np.nanmin(sss_int)
          smax = np.nanmax(sss_int)


          # ------------------------------------------------------------------------------------------------------------

          fig= plt.figure(figsize=(14,14))

          """Figure original SMOS at T0"""

          ax = fig.add_subplot(2,2,1,projection=ccrs.PlateCarree())
          map_plot(ax,lonmin,lonmax,latmin,latmax)
          pc = ax.pcolor(dsSMOS.sel(time=time_ini,method='nearest').lon.values,
                              dsSMOS.sel(time=time_ini,method='nearest').lat.values,
                              dsSMOS.sel(time=time_ini,method='nearest').SSS.values,vmin=31,vmax=37,cmap=plt.cm.jet)
          ax.set_title('SSS SMOS on ' + str(dsSMOS.sel(time=time_ini,method='nearest').time.values)[:10])
          ax.set_xlim(-60,-45)
          ax.set_ylim(-45,-30)

          """Plot the advected field at T0 (SMOS interpolated at particle position)"""

          ax = fig.add_subplot(2,2,2,projection=ccrs.PlateCarree())
          map_plot(ax,lonmin,lonmax,latmin,latmax)
          sc = ax.scatter(ds.lon[:,ind_ti].values,ds.lat[:,ind_ti].values,c=sss_int,vmin=31,vmax=37,cmap=plt.cm.jet,s=1.5)
          ax.set_title('SSS adv at Ti running time=' + filename[-11:-5] )  
          ax.set_xlim(-60,-45)
          ax.set_ylim(-45,-30)

          """Figure original SMOS at TF"""

          ax = fig.add_subplot(2,2,3,projection=ccrs.PlateCarree())
          map_plot(ax,lonmin,lonmax,latmin,latmax)
          pc = ax.pcolor(dsSMOS.sel(time=time_fin,method='nearest').lon.values,
                              dsSMOS.sel(time=time_fin,method='nearest').lat.values,
                              dsSMOS.sel(time=time_fin,method='nearest').SSS.values,vmin=31,vmax=37,cmap=plt.cm.jet)
          ax.set_title('SSS SMOS on ' + str(dsSMOS.sel(time=time_fin,method='nearest').time.values)[:10])
          ax.set_xlim(-60,-45)
          ax.set_ylim(-45,-30)

          """Plot the advected field at TF"""

          ax = fig.add_subplot(2,2,4,projection=ccrs.PlateCarree())
          map_plot(ax,lonmin,lonmax,latmin,latmax)
          sc = ax.scatter(ds.lon[:,ind_tf].values,ds.lat[:,ind_tf].values,c=sss_int,vmin=31,vmax=37,cmap=plt.cm.jet,s=.3)
          ax.set_title('SSS adv at Tf running time=' + filename[-11:-5])  
          ax.set_xlim(-60,-45)
          ax.set_ylim(-45,-30)
     
          if save == True:
               plt.savefig(pathPLOT + "advection" + '%02i'% rd +"days/" + filename[:18] + ".png", dpi=200,bbox_inches='tight',pad_inches = 0.1)

          plt.close()

          # ------------------------------------------------------------------------------------------------------------

          """Save dataset of advected sss"""

          name_file = pathADVSSS + "advection" + '%02i'% rd + "days/" + filename[:18] + "_sss.nc"

          if path.exists(name_file) == False:
               #shutil.copyfile(path_save+filename,name_file)
               print("")
               print("file for sss created, ... ", name_file)

               # Create variable
               ds = xr.Dataset(
                    data_vars = dict(
                         sss_adv = (["time"],sss_int)
                    ),
                    coords = dict(
                         lon = (["time"],ds.lon[:,ind_tf].values),
                         lat = (["time"],ds.lat[:,ind_tf].values),
                         time = ds.time[:,ind_tf].values
                    )
               )
               ds.to_netcdf(name_file)


Openning ... sim20160410_04days.zarr

ini date... 2016-04-06T00:00:00.000000000


Openning ... sim20160410_08days.zarr

ini date... 2016-04-02T00:00:00.000000000


Openning ... sim20160414_04days.zarr

ini date... 2016-04-10T00:00:00.000000000


Openning ... sim20160414_08days.zarr

ini date... 2016-04-06T00:00:00.000000000


Openning ... sim20160418_04days.zarr

ini date... 2016-04-14T00:00:00.000000000


Openning ... sim20160418_08days.zarr

ini date... 2016-04-10T00:00:00.000000000


Openning ... sim20160422_04days.zarr

ini date... 2016-04-18T00:00:00.000000000


Openning ... sim20160422_08days.zarr

ini date... 2016-04-14T00:00:00.000000000


Openning ... sim20160426_04days.zarr

ini date... 2016-04-22T00:00:00.000000000


Openning ... sim20160426_08days.zarr

ini date... 2016-04-18T00:00:00.000000000


Openning ... sim20160430_04days.zarr

ini date... 2016-04-26T00:00:00.000000000


Openning ... sim20160430_08days.zarr

ini date... 2016-04-22T00:00:00.000000000


Openning ... si